## Graph3: 🤖Bot with memory
- Memory & Checkpointers

In [3]:
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, START, END, add_messages
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.tools import TavilySearchResults
from langgraph.prebuilt import ToolNode
# from langchain_groq import ChatGroq
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
load_dotenv()
memory = MemorySaver()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
# llm = ChatGroq(model="llama-3.1-8b-instant")

class State(TypedDict):
    messages: Annotated[List, add_messages]

# tav_key = os.environ["TAVILY_API_KEY"]


search_tool = TavilySearchResults(max_results=4)
tools = [search_tool]

llm_with_tools = llm.bind_tools(tools=tools)

In [5]:
def chatbot(state:State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

def router(state:State):
    last_message = state["messages"][-1]

    if hasattr(last_message, "tool_calls") and len(last_message.tool_calls) > 0:
        return "tool_call"
    else: 
        return "end"

In [6]:
tool_node = ToolNode(tools=tools)

graph = StateGraph(State)
graph.add_node("chatbot", chatbot)
graph.add_node("tools", tool_node)

graph.set_entry_point("chatbot")
graph.add_conditional_edges("chatbot", router, {"tool_call":"tools", "end":END})
graph.add_edge("tools", "chatbot")

app = graph.compile(checkpointer=memory)
config = {"configurable":{
    "thread_id":1
}}

In [7]:
while True:
    user_input = input("👤 You: ")
    if user_input.lower() == "exit":
        print("👋 Bye Bye!")
        break
    else:
        
        result = app.invoke({"messages": [HumanMessage(content=user_input)]}, config=config)
        print(result)
        # print(f"BOT🤖: {result['messages'][-1].content}")

{'messages': [HumanMessage(content='Hi, I am Henry', additional_kwargs={}, response_metadata={}, id='fd78c958-e4c7-48f9-9923-0c63c59eb09e'), AIMessage(content='Hello Henry, how can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--eb59d211-8369-40b9-a449-60f49311a228-0', usage_metadata={'input_tokens': 54, 'output_tokens': 11, 'total_tokens': 65, 'input_token_details': {'cache_read': 0}})]}
{'messages': [HumanMessage(content='Hi, I am Henry', additional_kwargs={}, response_metadata={}, id='fd78c958-e4c7-48f9-9923-0c63c59eb09e'), AIMessage(content='Hello Henry, how can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--eb59d211-8369-40b9-a449-60f49311a228-0', usage_

In [8]:
# Checking the checkpoint state
print(app.get_state(config=config))

StateSnapshot(values={'messages': [HumanMessage(content='Hi, I am Henry', additional_kwargs={}, response_metadata={}, id='fd78c958-e4c7-48f9-9923-0c63c59eb09e'), AIMessage(content='Hello Henry, how can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--eb59d211-8369-40b9-a449-60f49311a228-0', usage_metadata={'input_tokens': 54, 'output_tokens': 11, 'total_tokens': 65, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='What is the current weather in Vaasa, Finland?', additional_kwargs={}, response_metadata={}, id='be428748-72d1-4ae1-9462-8d0156d4e204'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "current weather in Vaasa, Finland"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'ST